In [4]:
import pandas as pd
import numpy as np

from itertools import combinations

In [2]:
df = pd.read_csv('100kindex.csv')

In [3]:
df.head()

,Unnamed: 0,word,DF
0,0,striking,"{351: 1, 969: 1, 1111: 1, 3902: 1, 4049: 1, 52..."
1,1,anxiety,"{1458: 1, 2235: 1, 5341: 1, 8121: 1, 10582: 1,..."
2,2,ee,"{1: 2, 3: 8, 7: 16, 9: 1, 10: 10, 12: 4, 13: 2..."
3,3,forgave,"{17495: 3, 22451: 1, 24795: 1, 29181: 1, 29880..."
4,4,kick,"{287: 1, 628: 3, 959: 1, 1070: 1, 1589: 1, 164..."


In [ ]:
def get_candidate_resources(query=[]):
    CR = []
    out = False
    count = 0
    for doc in docs:
        if query in doc:
            count += 1
            CR.append(doc)
        if count == 50:
            break
    if count<50:
        combs = combinations(query, len(query)-1)
        for list(c) in list(combs):
            for doc in docs:
                if c in doc:
                    count+=1
                    CR.append()